# Librairies

In [1]:
import project_functions as pf
import pandas as pd

# Portfolio Analysis


Importons les données :

In [2]:
# récupérons les données des prix sauvegardés dans le fichier prices.csv
prices = pd.read_csv('datas/prices.csv', sep=',')
prices = prices.set_index('Date')

## Analyse des returns

Nous ne prenons uniquement les assets avec des returns anualisés positifs.

In [3]:
# Calculons les rendements journaliers
daily_returns = pf.daily_asset_return(prices)

# Calculons les rendements annuels
annualy_returns = pf.annualy_asset_return(daily_returns)

# On ne concerne uniquement les index avec rendements annuels positifs
positive_returns = annualy_returns[annualy_returns > 0]

# On récupère les tickers des index avec rendements positifs
tickers_pos = positive_returns.index.to_list()

Actualisons le dataframe prices avec uniquement les tickers avec un rendement positif

In [4]:
begin_train = "2014-01-01"
end_train = "2017-01-01"
begin_test = end_train
end_test = prices.index[-1]
prices_train, prices_test = pf.seperate_df(prices, begin_train, end_train, begin_test, end_test)

## Critères ESG

Nous allons créer notre portefeuille d'actions en se basant sur les critères ESG. Comme vu en cours cela s'appelle un ESG filter :
* exclude x% of firms with the worts ESG score
* keep firms with the best ESG momentum
* take a specific KPI

In [7]:
# récupérons les données liées aux critères ESG sauvegardés dans le fichier esg_scores.csv
esg = pd.read_csv('datas/esg_scores.csv', sep=',')

# supprimons les lignes tickers avec un return négatif
esg = esg[esg['Ticker Yahoo'].isin(tickers_vol)]

# On ne conserve que les tickers avec des scores ESG différents de NaN
esg = esg.dropna()

n = len(esg)

print(f'Nombre de tickers avec des scores ESG : {n}')

Nombre de tickers avec des scores ESG : 15


In [8]:
# On conserve les tickers avec un score environnemental supérieur à la médiane
esg = esg.sort_values(by=['Environment Score'])
tickers_env = esg.iloc[int(n*1/4):]
tickers_env

,A,Ticker Yahoo,Environment Score,Social Score,Governance Score,Total Score
51,51,SBMO.AS,47.0,83.0,85.0,72.0
60,60,VASTN.AS,52.0,45.0,74.0,57.0
42,42,NSI.AS,72.0,42.0,63.0,60.0
36,36,KPN.AS,77.0,76.0,67.0,74.0
13,13,BAMNB.AS,79.0,64.0,76.0,73.0
46,46,PNL.AS,79.0,82.0,74.0,79.0
45,45,PHIA.AS,81.0,94.0,84.0,88.0
49,49,REN.AS,82.0,84.0,95.0,87.0
3,3,AGN.AS,84.0,86.0,87.0,86.0
34,34,INGA.AS,86.0,68.0,89.0,78.0


In [9]:
# On conserve les tickers avec un score social supérieur à la médiane
esg = esg.sort_values(by=['Social Score'])
tickers_soc = esg.iloc[int(n*1/4):]
tickers_soc

,A,Ticker Yahoo,Environment Score,Social Score,Governance Score,Total Score
44,44,PHARM.AS,29.0,58.0,65.0,54.0
61,61,VLK.AS,21.0,61.0,51.0,50.0
13,13,BAMNB.AS,79.0,64.0,76.0,73.0
34,34,INGA.AS,86.0,68.0,89.0,78.0
36,36,KPN.AS,77.0,76.0,67.0,74.0
8,8,AMUND.AS,93.0,78.0,42.0,63.0
46,46,PNL.AS,79.0,82.0,74.0,79.0
51,51,SBMO.AS,47.0,83.0,85.0,72.0
49,49,REN.AS,82.0,84.0,95.0,87.0
3,3,AGN.AS,84.0,86.0,87.0,86.0


In [10]:
# Concaténation des dataframes en supprimant les doublons
tickers_esg = pd.merge(tickers_env, tickers_soc, how ='inner', on =['Ticker Yahoo'])
tickers_esg

,A_x,Ticker Yahoo,Environment Score_x,Social Score_x,Governance Score_x,Total Score_x,A_y,Environment Score_y,Social Score_y,Governance Score_y,Total Score_y
0,51,SBMO.AS,47.0,83.0,85.0,72.0,51,47.0,83.0,85.0,72.0
1,36,KPN.AS,77.0,76.0,67.0,74.0,36,77.0,76.0,67.0,74.0
2,13,BAMNB.AS,79.0,64.0,76.0,73.0,13,79.0,64.0,76.0,73.0
3,46,PNL.AS,79.0,82.0,74.0,79.0,46,79.0,82.0,74.0,79.0
4,45,PHIA.AS,81.0,94.0,84.0,88.0,45,81.0,94.0,84.0,88.0
5,49,REN.AS,82.0,84.0,95.0,87.0,49,82.0,84.0,95.0,87.0
6,3,AGN.AS,84.0,86.0,87.0,86.0,3,84.0,86.0,87.0,86.0
7,34,INGA.AS,86.0,68.0,89.0,78.0,34,86.0,68.0,89.0,78.0
8,52,SHELL.AS,92.0,93.0,95.0,93.0,52,92.0,93.0,95.0,93.0
9,8,AMUND.AS,93.0,78.0,42.0,63.0,8,93.0,78.0,42.0,63.0


In [11]:
# Récupérons la liste des tickers
list_esg = tickers_esg['Ticker Yahoo'].to_list()
print(f'Nombre de tickers ESG sélectionnés : {len(list_esg)}')

Nombre de tickers ESG sélectionnés : 10


## Enregistrement des tickers

Nous devons enregistrer les tickers sélectionnés dans le fichier tickers.txt

In [12]:
pf.tickers_storage(list_esg, 'datas/tickers.txt')